In [ ]:
import glob
import pickle
import numpy as np
import pandas as pd
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from pygame import *


In [ ]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=20, batch_size=64, callbacks=callbacks_list)

In [ ]:
model.summary()

In [ ]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Load the weights to each node
model.load_weights('weights-improvement-143-0.1657-bigger.hdf5', by_name=True)

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]

prediction_output = []

for note_index in range(300): #here, we're generating 300 notes
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)    
    
    prediction = model.predict(prediction_input, verbose=0)    
    index = np.argmax(prediction)

    result = int_to_note[index]
    prediction_output.append(result)    

    pattern.ravel()
    patternbeta =  pattern + index
    patternbeta = patternbeta[1:len(patternbeta)]

In [ ]:
offset = 0
output_notes = []

for pattern in prediction_output:
    #chords!
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = [] #creating the array where we'll store the note values, which the for loop below will handle
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes) #adding the note to the chord object
        new_chord.offset = offset #connecting it to the offset variable
        output_notes.append(new_chord) #adding it to the song
    #notes!
    else:
        new_note = note.Note(pattern) #storing it in the object
        new_note.offset = offset #connecting it to our offset command later on
        new_note.storedInstrument = instrument.Piano() #playing it with piano
        output_notes.append(new_note) #adding it to the song
    #make sure notes don't end up on top of each other by adding an 0.5 offset every time
    offset += 0.5

In [ ]:
s = stream.Stream(output_notes)

In [ ]:
mf = s.write('midi', fp="/Users/isaacnwang/desktop/Metis_Projects/Project_5/LofiGenerator/model2output.mid")